In [0]:
from pyspark.sql import functions as F # Importing functions from pyspark.sql
from pyspark.sql.functions import *
from pyspark.sql import Window
import pandas as pd

In [0]:
%run "/Workspace/Repos/yuan.niu@bayer.com/heme_new_writer_models_dev_repo/02_data_processing/helper_functions"

In [0]:
%run "../00_config/set-up"

In [0]:
hcp_demo = spark.sql("SELECT * FROM heme_data.hcp_demo")

In [0]:
hcp_affl = spark.sql("SELECT * FROM heme_data.hcp_affl")

In [0]:
acct_demo = spark.sql("SELECT * FROM heme_data.acct_demo")

In [0]:
acct_demo = get_data_snowflake(
f"""
  SELECT * FROM PHCDW.PHCDW_STG.MDM_STG_ACCT_PRFL
"""
)
print(acct_demo.count(), len(acct_demo.columns))

In [0]:
overlap_raw_data = spark.sql("SELECT * FROM heme_data.overlap_rx")
# Converting the original overlap data spark dataframe to pandas dataframe
""" Convert DecimalType columns to float to avoid UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [IU, PTD_FNL_CLM_AMT] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion."""
overlap_raw_data = overlap_raw_data.withColumn("IU", overlap_raw_data["IU"].cast("float"))
print('Row count: ', overlap_raw_data.count(), 'Column Count: ', len(overlap_raw_data.columns))

#### HCP Affiliation with institution flag

In [0]:
hcp_aff_wi_insn = (overlap_raw_data
                   .filter(col("INSN_NM").isNotNull())
                   .select("PTNT_ID",'INSN_NM').distinct()).cache()

In [0]:
display(hcp_aff_wi_insn.limit(20))

#### Size of HCP's affiliated account

In [0]:
hco_account_size = (overlap_raw_data.groupby(""))